In [ ]:
#### import global modules
import os
import sys
import pandas as pd
import numpy as np
from pathlib import Path
from yaml import safe_load
import google.oauth2.credentials
from google.cloud import bigquery
import gc

# Set global vars
pth_project = Path(os.getcwd().split('notebooks')[0])
pth_data = pth_project / 'data'
pth_queries = pth_project / 'core' / 'queries'
pth_creds = pth_project / 'conf' / 'local' / 'project_config.yaml'
sys.path.insert(0, str(pth_project))
d_project_config = safe_load(pth_creds.open())
# d_params = safe_load((pth_project / 'core' / 'parameters' / 'common.yaml').open())['data_extract']

# import local modules
from core.utils.gcp import connect_bq_services
# from core.etl.extract import extract_bq_data, extract_pr_codes, format_conv_df, filter_convs

# Connect to google services
bq_client = connect_bq_services(d_project_config['gcp-project-name'])
pd.options.display.max_rows = 100

In [ ]:
def extract_bq_data(bq_client, sql=None, pth_query=None):
    if sql is not None:
        df = bq_client.query(sql).to_dataframe()
    elif pth_query is not None:
        sql = pth_query.read_text()
        df = bq_client.query(sql).to_dataframe()
    else:
        raise ValueError('`sql` or `pth_query` should be set')  
    return df

In [ ]:
Query='''



with ADC_data_Sep_2022 as 

(

SELECT distinct
    customer_id,
    dealer_customer_id,
    BAN,
    CASE
      WHEN account_type_name='Standalone' THEN 'Smart_Camera'
      WHEN account_type_name='Awareness and Automation' THEN 'Smart_Automation_Plus'
    ELSE
    'Monitored'
  END
    AS Package,
    CASE
      WHEN Login_consistency = 0 THEN  "NO_User"
      WHEN Login_consistency>=30 THEN 'Heavy_User'
      WHEN Login_consistency<30 THEN "Low_User"
    ELSE
    'Un_assigned'
  END
    AS Smart_camera_Segment
   FROM
    `divgpras-pr-579355.ADC_Feature_Datastore.ADC_Master_Data`
  WHERE
    Month_Snapshot='2022-09-01'
    AND dealer_name='TELUS Communications Inc.' 
    
    
)


,ADC_data_Oct_2022 as (

SELECT distinct
    customer_id,
    dealer_customer_id,
    BAN,
    1 as Oct_data_flag,
    CASE
      WHEN account_type_name='Standalone' THEN 'Smart_Camera'
      WHEN account_type_name='Awareness and Automation' THEN 'Smart_Automation_Plus'
    ELSE
    'Monitored'
  END
    AS Package,
    CASE
      WHEN Login_consistency = 0 THEN  "NO_User"
      WHEN Login_consistency>=30 THEN 'Heavy_User'
      WHEN Login_consistency<30 THEN "Low_User"
    ELSE
    'Un_assigned'
  END
    AS Smart_camera_Segment
   FROM
    `divgpras-pr-579355.ADC_Feature_Datastore.ADC_Master_Data`
  WHERE
    Month_Snapshot='2022-10-01'
    AND dealer_name='TELUS Communications Inc.' 


)



,ADC_data_Nov_2022 as (

SELECT distinct
    customer_id,
    dealer_customer_id,
    BAN,
    1 as Nov_data_flag,
    CASE
      WHEN account_type_name='Standalone' THEN 'Smart_Camera'
      WHEN account_type_name='Awareness and Automation' THEN 'Smart_Automation_Plus'
    ELSE
    'Monitored'
  END
    AS Package,
    CASE
      WHEN Login_consistency = 0 THEN  "NO_User"
      WHEN Login_consistency>=30 THEN 'Heavy_User'
      WHEN Login_consistency<30 THEN "Low_User"
    ELSE
    'Un_assigned'
  END
    AS Smart_camera_Segment
   FROM
    `divgpras-pr-579355.ADC_Feature_Datastore.ADC_Master_Data`
  WHERE
    Month_Snapshot='2022-11-01'
    AND dealer_name='TELUS Communications Inc.' 


)


,ADC_data_Dec_2022 as (

SELECT distinct
    customer_id,
    dealer_customer_id,
    BAN,
    1 as Dec_data_flag,
    CASE
      WHEN account_type_name='Standalone' THEN 'Smart_Camera'
      WHEN account_type_name='Awareness and Automation' THEN 'Smart_Automation_Plus'
    ELSE
    'Monitored'
  END
    AS Package,
    CASE
      WHEN Login_consistency = 0 THEN  "NO_User"
      WHEN Login_consistency>=30 THEN 'Heavy_User'
      WHEN Login_consistency<30 THEN "Low_User"
    ELSE
    'Un_assigned'
  END
    AS Smart_camera_Segment
   FROM
    `divgpras-pr-579355.ADC_Feature_Datastore.ADC_Master_Data`
  WHERE
    Month_Snapshot='2022-12-01'
    AND dealer_name='TELUS Communications Inc.' 


)

,ADC_BASE as (
select a.*, b.Smart_camera_Segment as Smart_camera_Segment_Oct,b.Oct_data_flag,c.Smart_camera_Segment as Smart_camera_Segment_Nov,c.Nov_data_flag
,d.Smart_camera_Segment as Smart_camera_Segment_Dec,d.Dec_data_flag
from ADC_data_Sep_2022 a
left join ADC_data_Oct_2022 b
on a.customer_id=b.customer_id 
and a.dealer_customer_id=b.dealer_customer_id and a.BAN=b.BAN
left join ADC_data_Nov_2022 c
on a.customer_id=c.customer_id 
and a.dealer_customer_id=c.dealer_customer_id and a.BAN=c.BAN
left join ADC_data_Dec_2022 d
on a.customer_id=d.customer_id 
and a.dealer_customer_id=d.dealer_customer_id and a.BAN=d.BAN

)



,Telus_customers as(

select cust_bus_cust_id,pi_cntrct_start_ts as contract_start_date,pi_cntrct_end_ts as contract_end_date
from `cio-datahub-enterprise-pr-183a.ent_cust_cust.bq_prod_instnc_snpsht` 
WHERE DATE(prod_instnc_ts) = '2022-09-30' #Snapshot of the last day of the month
and pi_prod_instnc_typ_cd ='SMHM' #Serice type
and bus_prod_instnc_src_id = 1001 #BANs that are for home services
and pi_prod_instnc_stat_cd in ('A')
and  consldt_cust_typ_cd = 'R'
QUALIFY ROW_NUMBER() OVER (PARTITION BY cust_bus_cust_id ORDER BY prod_instnc_ts DESC) = 1
order by cust_bus_cust_id

)


,Merge_data as (


select a.*, b.contract_start_date,b.contract_end_date
from ADC_BASE  a
left join Telus_customers  b
on a.dealer_customer_id=b.cust_bus_cust_id


)


, Telus_SMHM_Deacts_Oct as


(

select distinct BILLING_ACCOUNT_NUM as BAN ,1 as Telus_Churn_Flag_Oct  

from `divgpras-pr-579355.SHS.SHS_DEACTS_OCT2022`

)

, Telus_SMHM_Deacts_Nov as


(

select distinct BILLING_ACCOUNT_NUM as BAN ,1 as Telus_Churn_Flag_Nov  

from `divgpras-pr-579355.SHS.SHS_DEACTS_NOV2022`

)


, Telus_SMHM_Deacts_Dec as


(

select distinct BILLING_ACCOUNT_NUM as BAN ,1 as Telus_Churn_Flag_Dec  

from `divgpras-pr-579355.SHS.SHS_DEACTS_DEC2022`

)


,Telus_SMHM_Deact_Nov2022 AS (


SELECT DISTINCT BAN,1 AS Telus_Churn_Flag_Nov_another
FROM `divg-churn-analysis-pr-7e40f6.divg_churn_analysis_pr_dataset.ad-hoc-2023-01-25` 

)




,Telus_Inactive_list as (


select bacct_bus_bacct_num as BAN,cust_bus_cust_id,pi_prod_instnc_stat_cd,prod_instnc_ts
from `cio-datahub-enterprise-pr-183a.ent_cust_cust.bq_prod_instnc_snpsht` 
WHERE DATE(prod_instnc_ts) < '2023-01-01'
and DATE(prod_instnc_ts) >= '2022-09-01'
and pi_prod_instnc_typ_cd ='SMHM' #Serice type
and bus_prod_instnc_src_id = 1001 #BANs that are for home services
and pi_prod_instnc_stat_cd in ('C')
and  consldt_cust_typ_cd = 'R'
QUALIFY ROW_NUMBER() OVER (PARTITION BY cust_bus_cust_id ORDER BY pi_prod_instnc_stat_ts DESC) = 1
order by prod_instnc_ts



)



,Final_data as (


select a.* ,b.Telus_Churn_Flag_Oct,c.Telus_Churn_Flag_Nov,d.Telus_Churn_Flag_Dec,e.Telus_Churn_Flag_Nov_another,f.prod_instnc_ts as Inactive_date_SHS
from Merge_data a
left join Telus_SMHM_Deacts_Oct b
on a.BAN=b.BAN
left join Telus_SMHM_Deacts_Nov c
on a.BAN=c.BAN
left join Telus_SMHM_Deacts_Dec d
on a.BAN=d.BAN
left join Telus_SMHM_Deact_Nov2022 e
on a.BAN=e.BAN
left join Telus_Inactive_list f
on a.BAN=f.BAN and a.dealer_customer_id=f.cust_bus_cust_id

)

select * from Final_data





'''

In [ ]:
# Query_SHS_Inacative='''

# select bacct_bus_bacct_num,cust_bus_cust_id,pi_prod_instnc_stat_cd,prod_instnc_ts
# from `cio-datahub-enterprise-pr-183a.ent_cust_cust.bq_prod_instnc_snpsht` 
# WHERE DATE(prod_instnc_ts) < '2023-01-01'
# and DATE(prod_instnc_ts) >= '2022-10-01'
# and pi_prod_instnc_typ_cd ='SMHM' #Serice type
# and bus_prod_instnc_src_id = 1001 #BANs that are for home services
# and pi_prod_instnc_stat_cd in ('C')
# and  consldt_cust_typ_cd = 'R'
# QUALIFY ROW_NUMBER() OVER (PARTITION BY cust_bus_cust_id ORDER BY pi_prod_instnc_stat_ts DESC) = 1
# order by prod_instnc_ts



# '''

In [ ]:
# DF_Inact=extract_bq_data(bq_client, sql=Query_SHS_Inacative)

In [ ]:
# DF_Inact.info()

In [ ]:
DF=extract_bq_data(bq_client, sql=Query)

In [ ]:
DF.info(verbose=True)

In [ ]:
# DF_1=DF.merge(

In [ ]:
DF.head()

In [ ]:
DF.isna().sum()*100/DF.shape[0]

In [ ]:
DF['Smart_camera_Segment_Oct'].fillna('Customer_not_found',inplace=True)
DF['Smart_camera_Segment_Nov'].fillna('Customer_not_found',inplace=True)
DF['Smart_camera_Segment_Dec'].fillna('Customer_not_found',inplace=True)

In [ ]:
DF['Telus_Churn_Flag_Oct'].fillna(0,inplace=True)
DF['Telus_Churn_Flag_Nov'].fillna(0,inplace=True)
DF['Telus_Churn_Flag_Dec'].fillna(0,inplace=True)
DF['Telus_Churn_Flag_Nov_another'].fillna(0,inplace=True)
DF['Inactive_date_SHS'].fillna('Active',inplace=True)



In [ ]:
DF['Inactive_date_SHS'].value_counts()

In [ ]:
DF['Telus_Churn_Flag_Nov_another'].value_counts()

In [ ]:
DF['Telus_Churn_Flag_Nov'].value_counts()

In [ ]:
DF['Telus_Churn_Flag_Dec'].value_counts()

In [ ]:
pd.DataFrame(DF[DF.Smart_camera_Segment=='NO_User'].groupby(['Smart_camera_Segment','Smart_camera_Segment_Oct','Smart_camera_Segment_Nov']).agg(
    
    Customer_count= ('customer_id','count'),
    Customer_count_1= ('customer_id','nunique'),
    Telus_Churn_Flag_Oct= ('Telus_Churn_Flag_Oct','sum'),
    Telus_Churn_Flag_Nov= ('Telus_Churn_Flag_Nov','sum'),
    Telus_Churn_Flag_Dec= ('Telus_Churn_Flag_Dec','sum')

).reset_index())

In [ ]:
pd.DataFrame(DF.groupby(['Smart_camera_Segment']).agg(
    
    Customer_count= ('customer_id','count'),
    Customer_count_1= ('customer_id','nunique'),
    Telus_Churn_Flag_Oct= ('Telus_Churn_Flag_Oct','sum'),
    Telus_Churn_Flag_Nov= ('Telus_Churn_Flag_Nov','sum'),
    Telus_Churn_Flag_Dec= ('Telus_Churn_Flag_Dec','sum')

).reset_index())

In [ ]:
pd.DataFrame(DF[DF.Package=='Smart_Camera'].groupby(['Smart_camera_Segment']).agg(
    
    Customer_count= ('customer_id','count'),
    Customer_count_1= ('customer_id','nunique'),
    Telus_Churn_Flag_Oct= ('Telus_Churn_Flag_Oct','sum'),
    Telus_Churn_Flag_Nov= ('Telus_Churn_Flag_Nov','sum'),
    Telus_Churn_Flag_Dec= ('Telus_Churn_Flag_Dec','sum')

).reset_index())

In [ ]:
pd.DataFrame(DF[(DF.Package=='Smart_Camera') & (DF.Smart_camera_Segment=='NO_User')].groupby(['Smart_camera_Segment','Smart_camera_Segment_Oct','Smart_camera_Segment_Nov']).agg(
    
    Customer_count= ('customer_id','count'),
    Customer_count_1= ('customer_id','nunique'),
    Telus_Churn_Flag_Oct= ('Telus_Churn_Flag_Oct','sum'),
    Telus_Churn_Flag_Nov= ('Telus_Churn_Flag_Nov','sum'),
    Telus_Churn_Flag_Dec= ('Telus_Churn_Flag_Dec','sum')

).reset_index())

In [ ]:
DF[(DF.Package=='Smart_Camera') & (DF.Smart_camera_Segment=='NO_User')].shape

In [ ]:
DF['Package'].value_counts()

In [ ]:
DF[(DF.Package=='Smart_Automation_Plus') & (DF.Smart_camera_Segment=='NO_User')].to_csv('Smart_Automation_Plus_No_users_Sep_MoM_analysis.csv',index=False)

In [ ]:
DF[(DF.Package=='Smart_Camera') & (DF.Smart_camera_Segment=='NO_User')].to_csv('Smart_Camera_No_users_Sep_MoM_analysis.csv',index=False)